In [1]:
import requests
import csv
import re
import json
import urllib.request
from bs4 import BeautifulSoup

for vano in range(2017, 2021):
    for vmes in range(1, 13):
        ano=str(vano)
        mes=""
        if vmes<10 :
            mes="0"         
        mes+=str(vmes)
        mesano=mes+ano
        anomes=ano+mes
        aux ="https://sisgvarmazenamento.blob.core.windows.net/prd/PublicacaoPortal/Arquivos/"
        aux+=anomes       
        aux+=".htm"
        #Consulte o site e retorne o html para a variável 'page'
        page = urllib.request.urlopen(aux)
        soup = BeautifulSoup(page, 'html5lib')


        # Criar um arquivo para gravar, adicionar linha de cabeçalhos
        c =open(('Dados'+mesano+'.csv'), 'w')
        f = csv.writer(c)
        f.writerow(['Vereador','Tipo_de_Gasto','Nome_Da_Empresa','CNPJ','Valor','Mes/Ano'])
        jsonFilePath=('Dados'+mesano+'.json')
        jsondata = [] 
       
        


    
        #Regex para remover tags html
        TAG_RE = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
        def remove_tags(text):
            return TAG_RE.sub('', text)

        #Inicialização de variaveis
        index=0
        start=0
        nomeVereador=""
        categoriaDespesa=""
        cnpj=""
        LugarDespesa=""
        skip=0
        ignore=0
        bugduplo=1
        #Percorre buscando Tds
        name_list = soup.find('body')
        name_list2 = soup.find(class_="bloco")
        for tr in name_list.find_all('tr'):
            if bugduplo==0:
                bugduplo=-1
                continue
            if(tr.find(text=re.compile("[\s\S]+(Vereador)\\((a)\\)[:\s]",re.I))):
                start=1
                name_list_itemsp = tr.find_all('td')
                for name in name_list_itemsp:
                    if(name.find(text=re.compile("[\s\S]+(Vereador)\\((a)\\)",re.I))):
                        names = str(name.contents[1])
                        names=re.sub("[\s\S]+(Vereador)\\((a)\\)[:\s]","",names)
                        names=remove_tags(names)
                        nomeVereador=names
                        ignore=0
                        if bugduplo==1:
                            bugduplo=0
            if(start!=0):
                name_list_itemsv = tr.find_all('td')
                for name in name_list_itemsv:                   
                    names = str(name.contents[0])
                    if(skip==1):
                        skip=0
                        continue
                    if(ignore==1):
                        continue
                    names=remove_tags(names)
                    names=re.sub("(Natureza da despesa)","",names)
                    names=re.sub("(Valor utilizado)","",names)                    
                    names=re.sub("(VALORES GASTOS)","",names)
                    names=re.sub("(VALORES DISPONIBILIZADOS)","",names)
                    names=re.sub("(TOTAL DO ITEM)","VXASkip",names)
                    names=re.sub("(TOTAL DO MÊS)","VXBSkip",names)
                    names=re.sub("(VEREADOR AFASTADO)","VXBSkip",names)
                    if re.match("\d{2}.?\d{3}.?\d{3}/?\d{4}-?\d{2}", names) is not None:              #RegexCNPJ
                        start=2
                    if re.match("[\s\S]*(VXASkip)", names) is not None:              #Corte Aqui  
                        start=1;
                        skip=1;
                        continue 
                    if re.match("[\s\S]*(VXBSkip)", names) is not None:              #Corte Aqui  
                        start=0;
                        ignore=1
                        break 
                    if re.match(r'^\s*$', names):
                        continue
                    if(start==1):
                        categoriaDespesa=names;
                        start=2;
                        continue            
                    if(start==2):   
                        cnpj=names;
                        start=3
                        continue
                    if(start==3):   
                        LugarDespesa=names;
                        start=4
                        continue    
                         #f = csv.writer(open(('Dados'+nomeVereador+'.csv'), 'w'))
                        #f.writerow(['Vereador','Tipo_de_Gasto','Nome_Da_Empresa','CNPJ','Valor','Mes/Ano'])
                    print(nomeVereador+", "+mes+"/"+ano)
                    #print(nomeVereador+", "+categoriaDespesa+", "+LugarDespesa+", "+cnpj+", "+names+", "+mes+"/"+ano)
                    if(start>=4):
                        #Codigo para vereador 
                        try:
                            t = open(('Dados'+nomeVereador+'.csv'), 'a')
                            g = csv.writer(t)
                        except:
                            t = open(('Dados'+nomeVereador+'.csv'), 'w')
                            g = csv.writer(t)
                            g.writerow(['Vereador','Tipo_de_Gasto','Nome_Da_Empresa','CNPJ','Valor','Mes/Ano'])
                        f.writerow([nomeVereador,categoriaDespesa,LugarDespesa,cnpj,names,(mes+"/"+ano)])    
                        g.writerow([nomeVereador,categoriaDespesa,LugarDespesa,cnpj,names,(mes+"/"+ano)]) 
                        t.close()
                        content={}
                        content['Vereador'] = nomeVereador
                        content['Tipo_de_Gasto'] = categoriaDespesa
                        content['Nome_Da_Empresa'] = LugarDespesa
                        content['CNPJ'] = cnpj
                        content['Valor'] = names
                        content['Mes/Ano'] = (mes+"/"+ano)
                        jsondata.append(content)
                        start=2;
                    start+=1
        #Ainda dentro do for de mes
        with open(jsonFilePath, 'w') as outfile:
            json.dump(jsondata, outfile)
        t.close()
        c.close()

 ABOU ANNI, 01/2017
 ABOU ANNI, 01/2017
 ABOU ANNI, 01/2017
 ABOU ANNI, 01/2017
 ADILSON AMADEU, 01/2017
 ADILSON AMADEU, 01/2017
 ADILSON AMADEU, 01/2017
 ADILSON AMADEU, 01/2017
 ADRIANA RAMALHO, 01/2017
 ADRIANA RAMALHO, 01/2017
 ADRIANA RAMALHO, 01/2017
 ADRIANA RAMALHO, 01/2017
 ADRIANA RAMALHO, 01/2017
 ALESSANDRO GUEDES, 01/2017
 ALESSANDRO GUEDES, 01/2017
 ALESSANDRO GUEDES, 01/2017
 ALESSANDRO GUEDES, 01/2017
 ALESSANDRO GUEDES, 01/2017
 ALESSANDRO GUEDES, 01/2017
 ALESSANDRO GUEDES, 01/2017
 ALESSANDRO GUEDES, 01/2017
 ALESSANDRO GUEDES, 01/2017
 ALESSANDRO GUEDES, 01/2017
 ALFREDINHO, 01/2017
 ALFREDINHO, 01/2017
 ALFREDINHO, 01/2017
 ALFREDINHO, 01/2017
 ALFREDINHO, 01/2017
 ALFREDINHO, 01/2017
 ALFREDINHO, 01/2017
 ALINE CARDOSO, 01/2017
 ALINE CARDOSO, 01/2017
 ANDRÉ SANTOS, 01/2017
 ANDRÉ SANTOS, 01/2017
 ARSELINO TATTO, 01/2017
 ARSELINO TATTO, 01/2017
 ARSELINO TATTO, 01/2017
 ARSELINO TATTO, 01/2017
 ARSELINO TATTO, 01/2017
 ARSELINO TATTO, 01/2017
 ARSELINO TATTO, 01

 AURELIO NOMURA, 02/2017
 AURELIO NOMURA, 02/2017
 AURELIO NOMURA, 02/2017
 AURELIO NOMURA, 02/2017
 AURELIO NOMURA, 02/2017
 CAIO MIRANDA, 02/2017
 CAIO MIRANDA, 02/2017
 CAIO MIRANDA, 02/2017
 CAIO MIRANDA, 02/2017
 CAIO MIRANDA, 02/2017
 CAMILO CRISTÓFARO, 02/2017
 CAMILO CRISTÓFARO, 02/2017
 CAMILO CRISTÓFARO, 02/2017
 CLAUDINHO DE SOUZA, 02/2017
 CLAUDINHO DE SOUZA, 02/2017
 CLAUDINHO DE SOUZA, 02/2017
 CLAUDINHO DE SOUZA, 02/2017
 CLAUDINHO DE SOUZA, 02/2017
 CLAUDINHO DE SOUZA, 02/2017
 CLAUDINHO DE SOUZA, 02/2017
 CLAUDINHO DE SOUZA, 02/2017
 CLAUDINHO DE SOUZA, 02/2017
 CLAUDIO FONSECA, 02/2017
 CLAUDIO FONSECA, 02/2017
 CLAUDIO FONSECA, 02/2017
 CONTE LOPES, 02/2017
 CONTE LOPES, 02/2017
 CONTE LOPES, 02/2017
 CONTE LOPES, 02/2017
 DALTON SILVANO, 02/2017
 DALTON SILVANO, 02/2017
 DALTON SILVANO, 02/2017
 DALTON SILVANO, 02/2017
 DALTON SILVANO, 02/2017
 DALTON SILVANO, 02/2017
 DALTON SILVANO, 02/2017
 DALTON SILVANO, 02/2017
 DAVID SOARES, 02/2017
 DAVID SOARES, 02/2017
 DA

 AURELIO NOMURA, 03/2017
 AURELIO NOMURA, 03/2017
 AURELIO NOMURA, 03/2017
 AURELIO NOMURA, 03/2017
 CAIO MIRANDA, 03/2017
 CAIO MIRANDA, 03/2017
 CAIO MIRANDA, 03/2017
 CAIO MIRANDA, 03/2017
 CAIO MIRANDA, 03/2017
 CAIO MIRANDA, 03/2017
 CAMILO CRISTÓFARO, 03/2017
 CAMILO CRISTÓFARO, 03/2017
 CAMILO CRISTÓFARO, 03/2017
 CAMILO CRISTÓFARO, 03/2017
 CAMILO CRISTÓFARO, 03/2017
 CAMILO CRISTÓFARO, 03/2017
 CAMILO CRISTÓFARO, 03/2017
 CLAUDINHO DE SOUZA, 03/2017
 CLAUDINHO DE SOUZA, 03/2017
 CLAUDINHO DE SOUZA, 03/2017
 CLAUDINHO DE SOUZA, 03/2017
 CLAUDINHO DE SOUZA, 03/2017
 CLAUDINHO DE SOUZA, 03/2017
 CLAUDIO FONSECA, 03/2017
 CLAUDIO FONSECA, 03/2017
 CLAUDIO FONSECA, 03/2017
 CLAUDIO FONSECA, 03/2017
 CLAUDIO FONSECA, 03/2017
 CLAUDIO FONSECA, 03/2017
 CLAUDIO FONSECA, 03/2017
 CLAUDIO FONSECA, 03/2017
 CONTE LOPES, 03/2017
 CONTE LOPES, 03/2017
 CONTE LOPES, 03/2017
 CONTE LOPES, 03/2017
 CONTE LOPES, 03/2017
 DALTON SILVANO, 03/2017
 DALTON SILVANO, 03/2017
 DALTON SILVANO, 03/2017

 ABOU ANNI, 04/2017
 ABOU ANNI, 04/2017
 ABOU ANNI, 04/2017
 ABOU ANNI, 04/2017
 ABOU ANNI, 04/2017
 ADILSON AMADEU, 04/2017
 ADILSON AMADEU, 04/2017
 ADRIANA RAMALHO, 04/2017
 ADRIANA RAMALHO, 04/2017
 ADRIANA RAMALHO, 04/2017
 ADRIANA RAMALHO, 04/2017
 ADRIANA RAMALHO, 04/2017
 ADRIANA RAMALHO, 04/2017
 ADRIANA RAMALHO, 04/2017
 ADRIANA RAMALHO, 04/2017
 ADRIANA RAMALHO, 04/2017
 ADRIANA RAMALHO, 04/2017
 ADRIANA RAMALHO, 04/2017
 ADRIANA RAMALHO, 04/2017
 ALESSANDRO GUEDES, 04/2017
 ALESSANDRO GUEDES, 04/2017
 ALESSANDRO GUEDES, 04/2017
 ALESSANDRO GUEDES, 04/2017
 ALESSANDRO GUEDES, 04/2017
 ALESSANDRO GUEDES, 04/2017
 ALESSANDRO GUEDES, 04/2017
 ALESSANDRO GUEDES, 04/2017
 ALESSANDRO GUEDES, 04/2017
 ALESSANDRO GUEDES, 04/2017
 ALFREDINHO, 04/2017
 ALFREDINHO, 04/2017
 ALFREDINHO, 04/2017
 ALFREDINHO, 04/2017
 ALFREDINHO, 04/2017
 ALFREDINHO, 04/2017
 ALFREDINHO, 04/2017
 ALFREDINHO, 04/2017
 ALFREDINHO, 04/2017
 ALINE CARDOSO, 04/2017
 ALINE CARDOSO, 04/2017
 ALINE CARDOSO, 04/20

 SANDRA TADEU, 04/2017
 SANDRA TADEU, 04/2017
 SANDRA TADEU, 04/2017
 SANDRA TADEU, 04/2017
 SANDRA TADEU, 04/2017
 SANDRA TADEU, 04/2017
 SANDRA TADEU, 04/2017
 SANDRA TADEU, 04/2017
 SENIVAL MOURA, 04/2017
 SENIVAL MOURA, 04/2017
 SENIVAL MOURA, 04/2017
 SENIVAL MOURA, 04/2017
 SENIVAL MOURA, 04/2017
 SENIVAL MOURA, 04/2017
 SONINHA, 04/2017
 SOUZA SANTOS, 04/2017
 SOUZA SANTOS, 04/2017
 SOUZA SANTOS, 04/2017
 SOUZA SANTOS, 04/2017
 SOUZA SANTOS, 04/2017
 SOUZA SANTOS, 04/2017
 SOUZA SANTOS, 04/2017
 SOUZA SANTOS, 04/2017
 SOUZA SANTOS, 04/2017
 SOUZA SANTOS, 04/2017
 SOUZA SANTOS, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO PAIVA, 04/2017
 TONINHO VESPOLI, 04/2017
 TONINHO VESPOLI, 04/2017
 TONINHO VESPOLI, 04/2017
 TONINHO VE

...

 PAULO FRANGE, 12/2019
 PAULO FRANGE, 12/2019
 QUITO FORMIGA, 12/2019
 QUITO FORMIGA, 12/2019
 QUITO FORMIGA, 12/2019
 QUITO FORMIGA, 12/2019
 QUITO FORMIGA, 12/2019
 QUITO FORMIGA, 12/2019
 QUITO FORMIGA, 12/2019
 QUITO FORMIGA, 12/2019
 QUITO FORMIGA, 12/2019
 QUITO FORMIGA, 12/2019
 REIS, 12/2019
 REIS, 12/2019
 REIS, 12/2019
 REIS, 12/2019
 REIS, 12/2019
 REIS, 12/2019
 REIS, 12/2019
 RICARDO NUNES, 12/2019
 RICARDO NUNES, 12/2019
 RICARDO NUNES, 12/2019
 RICARDO NUNES, 12/2019
 RICARDO NUNES, 12/2019
 RICARDO NUNES, 12/2019
 RICARDO NUNES, 12/2019
 RICARDO NUNES, 12/2019
 RICARDO TEIXEIRA, 12/2019
 RICARDO TEIXEIRA, 12/2019
 RICARDO TEIXEIRA, 12/2019
 RICARDO TEIXEIRA, 12/2019
 RICARDO TEIXEIRA, 12/2019
 RICARDO TEIXEIRA, 12/2019
 RINALDI DIGILIO, 12/2019
 RINALDI DIGILIO, 12/2019
 RINALDI DIGILIO, 12/2019
 RINALDI DIGILIO, 12/2019
 RINALDI DIGILIO, 12/2019
 RINALDI DIGILIO, 12/2019
 RINALDI DIGILIO, 12/2019
 RINALDI DIGILIO, 12/2019
 RINALDI DIGILIO, 12/2019
 RINALDI DIGILIO, 12/

 JAIR TATTO, 01/2020
 JAIR TATTO, 01/2020
 JAIR TATTO, 01/2020
 JAIR TATTO, 01/2020
 JAIR TATTO, 01/2020
 JAIR TATTO, 01/2020
 JAIR TATTO, 01/2020
 JAIR TATTO, 01/2020
 JAIR TATTO, 01/2020
 JANAINA LIMA, 01/2020
 JANAINA LIMA, 01/2020
 JOÃO JORGE, 01/2020
 JOÃO JORGE, 01/2020
 JOÃO JORGE, 01/2020
 JOÃO JORGE, 01/2020
 JOÃO JORGE, 01/2020
 JOSÉ POLICE NETO, 01/2020
 JOSÉ POLICE NETO, 01/2020
 JOSÉ POLICE NETO, 01/2020
 JOSÉ POLICE NETO, 01/2020
 JOSÉ POLICE NETO, 01/2020
 JULIANA CARDOSO, 01/2020
 JULIANA CARDOSO, 01/2020
 JULIANA CARDOSO, 01/2020
 JULIANA CARDOSO, 01/2020
 JULIANA CARDOSO, 01/2020
 JULIANA CARDOSO, 01/2020
 JULIANA CARDOSO, 01/2020
 JULIANA CARDOSO, 01/2020
 JULIANA CARDOSO, 01/2020
 JULIANA CARDOSO, 01/2020
 MÁRIO COVAS NETO, 01/2020
 MÁRIO COVAS NETO, 01/2020
 MÁRIO COVAS NETO, 01/2020
 MÁRIO COVAS NETO, 01/2020
 MÁRIO COVAS NETO, 01/2020
 MÁRIO COVAS NETO, 01/2020
 MÁRIO COVAS NETO, 01/2020
 MÁRIO COVAS NETO, 01/2020
 MÁRIO COVAS NETO, 01/2020
 MILTON FERREIRA, 01/2

 DONATO, 02/2020
 DONATO, 02/2020
 DONATO, 02/2020
 DONATO, 02/2020
 DONATO, 02/2020
 DONATO, 02/2020
 DONATO, 02/2020
 DONATO, 02/2020
 DONATO, 02/2020
 DONATO, 02/2020
 DONATO, 02/2020
 EDIR SALES, 02/2020
 EDIR SALES, 02/2020
 EDIR SALES, 02/2020
 EDIR SALES, 02/2020
 EDIR SALES, 02/2020
 EDIR SALES, 02/2020
 EDIR SALES, 02/2020
 EDIR SALES, 02/2020
 EDIR SALES, 02/2020
 EDIR SALES, 02/2020
 EDIR SALES, 02/2020
 EDIR SALES, 02/2020
 EDIR SALES, 02/2020
 EDUARDO MATARAZZO SUPLICY, 02/2020
 EDUARDO MATARAZZO SUPLICY, 02/2020
 EDUARDO MATARAZZO SUPLICY, 02/2020
 EDUARDO MATARAZZO SUPLICY, 02/2020
 EDUARDO MATARAZZO SUPLICY, 02/2020
 EDUARDO MATARAZZO SUPLICY, 02/2020
 EDUARDO MATARAZZO SUPLICY, 02/2020
 EDUARDO MATARAZZO SUPLICY, 02/2020
 EDUARDO MATARAZZO SUPLICY, 02/2020
 EDUARDO TUMA, 02/2020
 EDUARDO TUMA, 02/2020
 EDUARDO TUMA, 02/2020
 EDUARDO TUMA, 02/2020
 ELISEU GABRIEL, 02/2020
 ELISEU GABRIEL, 02/2020
 ELISEU GABRIEL, 02/2020
 ELISEU GABRIEL, 02/2020
 ELISEU GABRIEL, 02/2020

 AURELIO NOMURA, 03/2020
 AURELIO NOMURA, 03/2020
 AURELIO NOMURA, 03/2020
 AURELIO NOMURA, 03/2020
 CAIO MIRANDA, 03/2020
 CAIO MIRANDA, 03/2020
 CAIO MIRANDA, 03/2020
 CAIO MIRANDA, 03/2020
 CAMILO CRISTÓFARO, 03/2020
 CAMILO CRISTÓFARO, 03/2020
 CAMILO CRISTÓFARO, 03/2020
 CAMILO CRISTÓFARO, 03/2020
 CAMILO CRISTÓFARO, 03/2020
 CAMILO CRISTÓFARO, 03/2020
 CAMILO CRISTÓFARO, 03/2020
 CAMILO CRISTÓFARO, 03/2020
 CELSO GIANNAZI, 03/2020
 CELSO GIANNAZI, 03/2020
 CELSO GIANNAZI, 03/2020
 CELSO GIANNAZI, 03/2020
 CELSO GIANNAZI, 03/2020
 CELSO GIANNAZI, 03/2020
 CELSO GIANNAZI, 03/2020
 CELSO GIANNAZI, 03/2020
 CLAUDINHO DE SOUZA, 03/2020
 CLAUDINHO DE SOUZA, 03/2020
 CLAUDINHO DE SOUZA, 03/2020
 CLAUDINHO DE SOUZA, 03/2020
 CLAUDIO FONSECA, 03/2020
 CLAUDIO FONSECA, 03/2020
 CLAUDIO FONSECA, 03/2020
 CLAUDIO FONSECA, 03/2020
 CLAUDIO FONSECA, 03/2020
 CLAUDIO FONSECA, 03/2020
 CLAUDIO FONSECA, 03/2020
 CLAUDIO FONSECA, 03/2020
 CLAUDIO FONSECA, 03/2020
 CLAUDIO FONSECA, 03/2020
 CLAUDIO

 ADILSON AMADEU, 04/2020
 ADILSON AMADEU, 04/2020
 ADRIANA RAMALHO, 04/2020
 ADRIANA RAMALHO, 04/2020
 ADRIANA RAMALHO, 04/2020
 ADRIANA RAMALHO, 04/2020
 ADRIANA RAMALHO, 04/2020
 ADRIANA RAMALHO, 04/2020
 ADRIANA RAMALHO, 04/2020
 ADRIANA RAMALHO, 04/2020
 ADRIANA RAMALHO, 04/2020
 ADRIANA RAMALHO, 04/2020
 ADRIANA RAMALHO, 04/2020
 ADRIANA RAMALHO, 04/2020
 ALESSANDRO GUEDES, 04/2020
 ALESSANDRO GUEDES, 04/2020
 ALESSANDRO GUEDES, 04/2020
 ALESSANDRO GUEDES, 04/2020
 ALESSANDRO GUEDES, 04/2020
 ALESSANDRO GUEDES, 04/2020
 ALESSANDRO GUEDES, 04/2020
 ALESSANDRO GUEDES, 04/2020
 ALFREDINHO, 04/2020
 ALFREDINHO, 04/2020
 ALFREDINHO, 04/2020
 ALFREDINHO, 04/2020
 ALFREDINHO, 04/2020
 ANDRÉ SANTOS, 04/2020
 ANDRÉ SANTOS, 04/2020
 ANDRÉ SANTOS, 04/2020
 ANDRÉ SANTOS, 04/2020
 ANDRÉ SANTOS, 04/2020
 ARSELINO TATTO, 04/2020
 ARSELINO TATTO, 04/2020
 ARSELINO TATTO, 04/2020
 ARSELINO TATTO, 04/2020
 ARSELINO TATTO, 04/2020
 ARSELINO TATTO, 04/2020
 ATILIO FRANCISCO, 04/2020
 ATILIO FRANCISCO